# Pregunta 1 — TWFE & Event-Study (R)

## Librerías

## Instalación de paquetes (ejecutar una sola vez)

In [53]:
# Instalar paquetes necesarios (ejecutar UNA sola vez)
#install.packages("dplyr", repos = "https://cloud.r-project.org")
#install.packages("tidyr", repos = "https://cloud.r-project.org")
#install.packages("data.table", repos = "https://cloud.r-project.org")
#install.packages("fixest", repos = "https://cloud.r-project.org")
#nstall.packages("ggplot2", repos = "https://cloud.r-project.org")

#cat("\n✓ Instalación completada!\n")

In [54]:
# =============================================================================
# SETUP: Load Required Libraries
# =============================================================================
library(tidyverse)
library(fixest)
library(broom)
library(knitr)
library(scales)

theme_set(theme_minimal(base_size = 14))
cat("Libraries loaded successfully!\n")

✓ Librerías cargadas exitosamente!


## Carga de Datos

In [55]:
url <- "https://raw.githubusercontent.com/LOST-STATS/LOST-STATS.github.io/master/Model_Estimation/Data/Event_Study_DiD/bacon_example.csv"
df <- read.csv(url)

# Normalize column names
if ('_nfd' %in% names(df)) df <- df %>% rename(X_nfd = `_nfd`)

# Create treatment indicator (ever-treated)
df <- df %>% mutate(X_treated = ifelse(!is.na(X_nfd) & X_nfd > 0, 1, 0))

cat("Dataset loaded:", nrow(df), "rows x", ncol(df), "columns\n")
cat("States:", length(unique(df$stfips)), "| Years:", min(df$year), "-", max(df$year), "\n")
head(df, 6)

Dataset Shape: 1617 x 10 

Primeras filas:
    stfips  year  _nfd  post    asmrs    pcinc    asmrh       cases  weight
     <int> <int> <int> <int>    <num>    <num>    <num>       <num>   <num>
 1:      1  1964  1971     0 35.63988 12406.18 5.007341 0.012312243 1715156
 2:      1  1965  1971     0 41.54375 13070.21 4.425367 0.010419407 1715156
 3:      1  1966  1971     0 34.25233 13526.66 4.874819 0.009900097 1715156
 4:      1  1967  1971     0 34.46502 13918.19 5.362014 0.009974692 1715156
 5:      1  1968  1971     0 40.44011 14684.81 4.643759 0.012400659 1715156
 6:      1  1969  1971     0 42.49012 15638.88 5.296976 0.015006756 1715156
 7:      1  1970  1971     0 44.23907 16222.95 4.716939 0.019703142 1715156
 8:      1  1971  1971     1 49.91467 16875.43 4.992504 0.022737451 1715156
 9:      1  1972  1971     1 47.20313 17976.76 5.187023 0.023555981 1715156
10:      1  1973  1971     1 41.03307 18808.25 5.085546 0.025031525 1715156
      copop
      <num>
 1: 1715156
 2: 17251

     stfips          year           _nfd           post       
 Min.   : 1.0   Min.   :1964   Min.   :1969   Min.   :0.0000  
 1st Qu.:18.0   1st Qu.:1972   1st Qu.:1971   1st Qu.:0.0000  
 Median :30.0   Median :1980   Median :1973   Median :1.0000  
 Mean   :29.8   Mean   :1980   Mean   :1974   Mean   :0.6846  
 3rd Qu.:42.0   3rd Qu.:1988   3rd Qu.:1974   3rd Qu.:1.0000  
 Max.   :56.0   Max.   :1996   Max.   :1985   Max.   :1.0000  
                               NA's   :429                    
     asmrs             pcinc           asmrh            cases         
 Min.   :  7.477   Min.   :10275   Min.   : 0.000   Min.   :0.003131  
 1st Qu.: 40.238   1st Qu.:21462   1st Qu.: 2.166   1st Qu.:0.015990  
 Median : 48.842   Median :25783   Median : 3.207   Median :0.023150  
 Mean   : 52.166   Mean   :26081   Mean   : 3.439   Mean   :0.024491  
 3rd Qu.: 60.036   3rd Qu.:30008   3rd Qu.: 4.356   3rd Qu.:0.031429  
 Max.   :185.971   Max.   :48822   Max.   :19.134   Max.   :0.088966  

## a) TWFE - Regresión de Efectos Fijos Bidireccionales

**Especificación:**
$$Y_{it} = \alpha_i + \gamma_t + \beta \cdot D_{it} + X_{it}'\delta + \epsilon_{it}$$

Donde:
- $Y_{it}$ = asmrs (tasa de mortalidad ajustada por edad)
- $\alpha_i$ = efectos fijos por estado (stfips)
- $\gamma_t$ = efectos fijos por año (year)
- $D_{it}$ = indicador de tratamiento post (treated_post)
- $X_{it}$ = controles (pcinc, asmrh, cases)

### Exploración de la Estructura de Tratamiento

In [56]:
cat("Años de adopción del tratamiento (_nfd):\n")
print(sort(unique(df$`_nfd`)))
cat("\nNúmero de unidades (estados):", df$stfips %>% n_distinct(), "\n")
cat("Rango temporal:", min(df$year), "-", max(df$year), "\n\n")
cat("Distribución por status de tratamiento:\n")
print(table(df$`_nfd`))

Años de adopción del tratamiento (_nfd):
 [1] 1969 1970 1971 1972 1973 1974 1975 1976 1977 1980 1984 1985

Número de unidades (estados): 49 
 [1] 1969 1970 1971 1972 1973 1974 1975 1976 1977 1980 1984 1985

Número de unidades (estados): 49 
Rango temporal: 1964 - 1996 

Distribución por status de tratamiento:

1969 1970 1971 1972 1973 1974 1975 1976 1977 1980 1984 1985 
  66   66  231   99  330   99   66   33   99   33   33   33 
Rango temporal: 1964 - 1996 

Distribución por status de tratamiento:

1969 1970 1971 1972 1973 1974 1975 1976 1977 1980 1984 1985 
  66   66  231   99  330   99   66   33   99   33   33   33 


### Estimación TWFE

In [57]:
df <- df %>%
  mutate(
    `_treated` = as.integer(`_nfd` > 0),
    post = as.integer(`_nfd` > 0 & year >= `_nfd`),
    treated_post = post * `_treated`
  )

twfe_model <- feols(
  asmrs ~ treated_post + pcinc + asmrh + cases | stfips + year,
  data = df,
  cluster = "stfips"
)

cat(paste(rep("=", 80), collapse = ""), "\n")
cat("REGRESIÓN TWFE - RESULTADOS\n")
cat(paste(rep("=", 80), collapse = ""), "\n")
print(twfe_model)

twfe_coef <- coef(twfe_model)["treated_post"]
twfe_se <- se(twfe_model)["treated_post"]

summary_twfe <- tibble(
  Variable = names(coef(twfe_model)),
  Coeficiente = coef(twfe_model),
  `Error Std.` = se(twfe_model),
  `t-stat` = coef(twfe_model) / se(twfe_model),
  `p-valor` = 2 * (1 - pnorm(abs(coef(twfe_model) / se(twfe_model)))),
  `IC 95% Inferior` = coef(twfe_model) - 1.96 * se(twfe_model),
  `IC 95% Superior` = coef(twfe_model) + 1.96 * se(twfe_model)
)

cat("\n", paste(rep("=", 80), collapse = ""), "\n")
cat("TABLA RESUMEN TWFE\n")
cat(paste(rep("=", 80), collapse = ""), "\n")
print(as.data.frame(summary_twfe), row.names = FALSE)
cat("\n*** Efecto del Tratamiento (treated_post):", sprintf("%.4f", twfe_coef), 
    "(SE:", sprintf("%.4f", twfe_se), ") ***\n")

NOTE: 429 observations removed because of NA values (RHS: 429).



REGRESIÓN TWFE - RESULTADOS
OLS estimation, Dep. Var.: asmrs
Observations: 1,188
Fixed-effects: stfips: 36,  year: 33
Standard-errors: Clustered (stfips) 
               Estimate Std. Error   t value Pr(>|t|)    
treated_post   2.475811   3.024245  0.818654 0.418523    
pcinc         -0.000726   0.000417 -1.740622 0.090536 .  
asmrh          0.508696   0.371010  1.371113 0.179068    
cases        -96.324096 146.887959 -0.655766 0.516263    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 10.5     Adj. R2: 0.736853
             Within R2: 0.012412

REGRESIÓN TWFE - RESULTADOS
OLS estimation, Dep. Var.: asmrs
Observations: 1,188
Fixed-effects: stfips: 36,  year: 33
Standard-errors: Clustered (stfips) 
               Estimate Std. Error   t value Pr(>|t|)    
treated_post   2.475811   3.024245  0.818654 0.418523    
pcinc         -0.000726   0.000417 -1.740622 0.090536 .  
asmrh          0.508696   0.371010  1.371113 0.179068    
cases        -96.324096 146.887959

## b) Preparación para Event-Study

### Paso 1: Crear la variable de tiempo relativo

In [58]:
df_es <- df %>%
  mutate(
    treatment_year = `_nfd`,
    event_time = if_else(treatment_year > 0, year - treatment_year, NA_integer_)
  )

cat("Variable de tiempo relativo creada!\n\n")
cat("Muestra de datos con event_time:\n")
print(head(df_es %>% select(stfips, year, treatment_year, event_time, asmrs), 15))

Variable de tiempo relativo creada!

Muestra de datos con event_time:
    stfips  year treatment_year event_time    asmrs
     <int> <int>          <int>      <int>    <num>
 1:      1  1964           1971         -7 35.63988
 2:      1  1965           1971         -6 41.54375
 3:      1  1966           1971         -5 34.25233
 4:      1  1967           1971         -4 34.46502
 5:      1  1968           1971         -3 40.44011
 6:      1  1969           1971         -2 42.49012
 7:      1  1970           1971         -1 44.23907
 8:      1  1971           1971          0 49.91467
 9:      1  1972           1971          1 47.20313
10:      1  1973           1971          2 41.03307
11:      1  1974           1971          3 46.20281
12:      1  1975           1971          4 39.24017
13:      1  1976           1971          5 48.39191
14:      1  1977           1971          6 40.51945
15:      1  1978           1971          7 33.85831
Muestra de datos con event_time:
    stfips  y

### Paso 2: Tabla de frecuencias del tiempo relativo

In [59]:
event_time_freq <- df_es %>%
  filter(!is.na(event_time)) %>%
  count(event_time, name = "Frecuencia") %>%
  arrange(event_time) %>%
  mutate(
    Porcentaje = round(100 * Frecuencia / sum(Frecuencia), 2),
    `Porcentaje Acum.` = cumsum(Porcentaje)
  )

cat(paste(rep("=", 70), collapse = ""), "\n")
cat("TABLA DE FRECUENCIAS - TIEMPO RELATIVO AL TRATAMIENTO\n")
cat(paste(rep("=", 70), collapse = ""), "\n")
# Usar as.data.frame() para evitar dispatch a print.data.table con argumento `n`
print(as.data.frame(event_time_freq), row.names = FALSE)
cat("\nTotal de observaciones con event_time:", sum(event_time_freq$Frecuencia), "\n")
cat("Observaciones nunca tratadas:", sum(is.na(df_es$event_time)), "\n")
cat("Rango de tiempo relativo:", min(event_time_freq$event_time), "a", max(event_time_freq$event_time), "\n")

write.csv(event_time_freq, "../output/salida_1.csv", row.names = FALSE)
cat("\n✓ Tabla de frecuencias guardada en: ../output/salida_1.csv\n")

TABLA DE FRECUENCIAS - TIEMPO RELATIVO AL TRATAMIENTO
 event_time Frecuencia Porcentaje Porcentaje Acum.
        -21          1       0.08             0.08
        -20          2       0.17             0.25
        -19          2       0.17             0.42
        -18          2       0.17             0.59
        -17          2       0.17             0.76
        -16          3       0.25             1.01
        -15          3       0.25             1.26
        -14          3       0.25             1.51
        -13          6       0.51             2.02
        -12          7       0.59             2.61
        -11          9       0.76             3.37
        -10         12       1.01             4.38
         -9         22       1.85             6.23
         -8         25       2.10             8.33
         -7         32       2.69            11.02
         -6         34       2.86            13.88
         -5         36       3.03            16.91
         -4         36      

### Paso 3: Selección de límites para agrupar extremos

In [60]:
valid_times <- df_es %>%
  filter(!is.na(event_time)) %>%
  pull(event_time)

cat(paste(rep("=", 70), collapse = ""), "\n")
cat("ANÁLISIS DE PERCENTILES - SELECCIÓN DE LÍMITES\n")
cat(paste(rep("=", 70), collapse = ""), "\n")

percentiles <- c(0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99)
perc_values <- quantile(valid_times, probs = percentiles, na.rm = TRUE)
for (i in seq_along(percentiles)) {
  cat(sprintf("Percentil %5.0f%%: %6.1f\n", percentiles[i] * 100, perc_values[i]))
}

cat("\nESTADÍSTICOS DESCRIPTIVOS:\n")
cat("Mínimo:  ", min(valid_times), "\n")
cat("Máximo:  ", max(valid_times), "\n")
cat("Media:   ", round(mean(valid_times), 2), "\n")
cat("Mediana: ", median(valid_times), "\n")
cat("D.E.:    ", round(sd(valid_times), 2), "\n")

LOWER_BOUND <- quantile(valid_times, probs = 0.10, na.rm = TRUE) %>% as.integer()
UPPER_BOUND <- quantile(valid_times, probs = 0.90, na.rm = TRUE) %>% as.integer()

cat("\n>>> Límites seleccionados (basados en percentiles 10%-90%):\n")
cat("LOWER_BOUND = ", LOWER_BOUND, "\n")
cat("UPPER_BOUND = ", UPPER_BOUND, "\n")
cat(paste(rep("=", 70), collapse = ""), "\n")

ANÁLISIS DE PERCENTILES - SELECCIÓN DE LÍMITES
Percentil     1%:  -15.1
Percentil     5%:   -9.0
Percentil    10%:   -7.0
Percentil    25%:   -2.0
Percentil    50%:    6.0
Percentil    75%:   15.0
Percentil    90%:   20.0
Percentil    95%:   22.0
Percentil    99%:   25.0

ESTADÍSTICOS DESCRIPTIVOS:
Mínimo:   -21 
Máximo:   27 
Media:    6.42 
Mediana:  6 
ANÁLISIS DE PERCENTILES - SELECCIÓN DE LÍMITES
Percentil     1%:  -15.1
Percentil     5%:   -9.0
Percentil    10%:   -7.0
Percentil    25%:   -2.0
Percentil    50%:    6.0
Percentil    75%:   15.0
Percentil    90%:   20.0
Percentil    95%:   22.0
Percentil    99%:   25.0

ESTADÍSTICOS DESCRIPTIVOS:
Mínimo:   -21 
Máximo:   27 
Media:    6.42 
Mediana:  6 
D.E.:     10.16 

>>> Límites seleccionados (basados en percentiles 10%-90%):
LOWER_BOUND =  -7 
UPPER_BOUND =  20 
D.E.:     10.16 

>>> Límites seleccionados (basados en percentiles 10%-90%):
LOWER_BOUND =  -7 
UPPER_BOUND =  20 


### Paso 4: Creación de variables dummy para el time relativo

In [61]:
cat("=== VERIFICACIÓN PRE-DUMMY ===\n")
cat("LOWER_BOUND:", class(LOWER_BOUND), "=", LOWER_BOUND, "\n")
cat("UPPER_BOUND:", class(UPPER_BOUND), "=", UPPER_BOUND, "\n")
cat("Rango de loop: ", LOWER_BOUND, ":", UPPER_BOUND, "\n")
cat("Secuencia resultante:\n")
print(LOWER_BOUND:UPPER_BOUND)
cat("\n")

df_es <- df_es %>%
  mutate(
    event_time_binned = case_when(
      is.na(event_time) ~ NA_integer_,
      event_time < LOWER_BOUND ~ LOWER_BOUND,
      event_time > UPPER_BOUND ~ UPPER_BOUND,
      TRUE ~ event_time
    )
  )

dummy_cols <- c()
for (t in LOWER_BOUND:UPPER_BOUND) {
  col_name <- paste0("d_", t)
  cat("Creando columna:", col_name, "\n")
  df_es <- df_es %>%
    mutate(
      !!col_name := if_else(!is.na(event_time_binned) & event_time_binned == t, 1L, 0L)
    )
  dummy_cols <- c(dummy_cols, col_name)
}

cat("\n")
cat(paste(rep("=", 70), collapse = ""), "\n")
cat("DUMMIES DE TIEMPO RELATIVO CREADAS\n")
cat(paste(rep("=", 70), collapse = ""), "\n")
cat("Total de dummies:", length(dummy_cols), "\n")
cat("Rango de tiempo relativo binned:", LOWER_BOUND, "a", UPPER_BOUND, "\n")
cat("\nDummies creadas:\n")
for (i in 1:length(dummy_cols)) {
  if (i %% 5 == 1) cat("\n")
  cat(sprintf("%-8s", dummy_cols[i]))
}
cat("\n")

dummy_sums <- df_es %>% select(all_of(dummy_cols)) %>% colSums()
cat("\nFrecuencias de dummies (primeros 10):\n")
for (i in 1:min(10, length(dummy_cols))) {
  cat(sprintf("%s: %4d  ", dummy_cols[i], dummy_sums[i]))
  if (i %% 5 == 0) cat("\n")
}
cat("\n")

# Excluir d_-1 como referencia y filtrar elementos vacíos
dummy_cols_for_reg <- dummy_cols[dummy_cols != "d_-1"]
dummy_cols_for_reg <- dummy_cols_for_reg[nchar(dummy_cols_for_reg) > 0 & !is.na(dummy_cols_for_reg)]

cat("\nDummies para la regresión (excluyendo d_-1 como referencia):", length(dummy_cols_for_reg), "\n")
cat("Lista de dummies:\n")
print(dummy_cols_for_reg)
cat("\n")
cat(paste(rep("=", 70), collapse = ""), "\n")

=== VERIFICACIÓN PRE-DUMMY ===
LOWER_BOUND: integer = -7 
UPPER_BOUND: integer = 20 
Rango de loop:  -7 : 20 
Secuencia resultante:
 [1] -7 -6 -5 -4 -3 -2 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17
[26] 18 19 20

LOWER_BOUND: integer = -7 
UPPER_BOUND: integer = 20 
Rango de loop:  -7 : 20 
Secuencia resultante:
 [1] -7 -6 -5 -4 -3 -2 -1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17
[26] 18 19 20

Creando columna: d_-7 
Creando columna: d_-6 
Creando columna: d_-5 
Creando columna: d_-4 
Creando columna: d_-3 
Creando columna: d_-2 
Creando columna: d_-1 
Creando columna: d_0 
Creando columna: d_1 
Creando columna: d_2 
Creando columna: d_3 
Creando columna: d_4 
Creando columna: d_5 
Creando columna: d_6 
Creando columna: d_7 
Creando columna: d_8 
Creando columna: d_9 
Creando columna: d_10 
Creando columna: d_11 
Creando columna: d_12 
Creando columna: d_13 
Creando columna: d_14 
Creando columna: d_15 
Creando columna: d_16 
Creando columna: d_17 
Creando colu

## c) Estimación del Event-Study

In [62]:
# DEBUG: Verificar las variables disponibles
cat("=== DEBUG - VERIFICACIÓN DE FÓRMULA ===\n")
cat("dummy_cols_for_reg contiene", length(dummy_cols_for_reg), "variables\n")
cat("Primeras 5:\n")
print(head(dummy_cols_for_reg, 5))
cat("\nÚltimas 5:\n")
print(tail(dummy_cols_for_reg, 5))

# Crear la fórmula manualmente para asegurar que sea correcta
cat("\n--- Construcción de fórmula ---\n")
formula_rhs <- paste(dummy_cols_for_reg, collapse = " + ")

# Verificar que no haya "d_" vacío
if (grepl("d_ ", formula_rhs) || grepl(" d_$", formula_rhs) || grepl("d_\\+", formula_rhs)) {
  cat("❌ ERROR: Hay elementos vacíos en la fórmula!\n")
  cat("RHS:\n")
  cat(formula_rhs, "\n")
  cat("\nLimpiando...\n")
  # Filtrar nuevamente
  dummy_cols_for_reg <- dummy_cols_for_reg[nchar(dummy_cols_for_reg) > 2]
  formula_rhs <- paste(dummy_cols_for_reg, collapse = " + ")
}

formula_str <- paste("asmrs ~", formula_rhs, "| stfips + year", sep = " ")

cat("\n✓ Fórmula final construida:\n")
cat(formula_str, "\n")
cat("\n--- Verificación de variables ---\n")
cat("¿Todas las variables están en df_es?\n")

# Extraer nombres de variables de la fórmula
vars_in_formula <- unique(c(
  dummy_cols_for_reg,
  "asmrs", "stfips", "year"
))

missing <- setdiff(vars_in_formula, names(df_es))
if (length(missing) > 0) {
  cat("❌ Faltan variables:\n")
  print(missing)
  stop("No puedo continuar sin estas variables")
} else {
  cat("✓ ¡Todas las variables están presentes en df_es!\n")
}

cat("=== FIN DEBUG ===\n\n")

es_model <- feols(
  as.formula(formula_str),
  data = df_es,
  cluster = "stfips"
)

cat("\n")
print(es_model)

# Extraer coeficientes y errores estándar con funciones apropiadas
coef_vec <- coef(es_model)
se_vec <- se(es_model)

# Crear tibble con resultados
es_coef <- tibble(
  term = names(coef_vec),
  estimate = as.numeric(coef_vec),
  se = as.numeric(se_vec)
) %>%
  mutate(
    t_stat = estimate / se,
    p_value = 2 * (1 - pnorm(abs(t_stat))),
    ci_lower = estimate - 1.96 * se,
    ci_upper = estimate + 1.96 * se,
    significant = case_when(
      p_value < 0.01 ~ "***",
      p_value < 0.05 ~ "**",
      p_value < 0.10 ~ "*",
      TRUE ~ ""
    )
  )

# Filtrar solo variables d_* y extraer event_time
es_results_df <- es_coef %>%
  filter(grepl("^d_", term)) %>%
  mutate(
    event_time = as.integer(gsub("^d_", "", term))
  ) %>%
  select(event_time, estimate, se, t_stat, p_value, ci_lower, ci_upper, significant) %>%
  arrange(event_time)

cat("\n")
cat(paste(rep("=", 85), collapse = ""), "\n")
cat("RESULTADOS DEL EVENT-STUDY - EFECTOS DINÁMICOS POR TIEMPO RELATIVO\n")
cat(paste(rep("=", 85), collapse = ""), "\n")
print(as.data.frame(es_results_df), row.names = FALSE)

write.csv(es_results_df, "../output/salida_2.csv", row.names = FALSE)
cat("\n✓ Resultados del event-study guardados en: ../output/salida_2.csv\n")

=== DEBUG - VERIFICACIÓN DE FÓRMULA ===
dummy_cols_for_reg contiene 27 variables
Primeras 5:
[1] "d_-7" "d_-6" "d_-5" "d_-4" "d_-3"

Últimas 5:
dummy_cols_for_reg contiene 27 variables
Primeras 5:
[1] "d_-7" "d_-6" "d_-5" "d_-4" "d_-3"

Últimas 5:
[1] "d_16" "d_17" "d_18" "d_19" "d_20"

--- Construcción de fórmula ---

✓ Fórmula final construida:
asmrs ~ d_-7 + d_-6 + d_-5 + d_-4 + d_-3 + d_-2 + d_0 + d_1 + d_2 + d_3 + d_4 + d_5 + d_6 + d_7 + d_8 + d_9 + d_10 + d_11 + d_12 + d_13 + d_14 + d_15 + d_16 + d_17 + d_18 + d_19 + d_20 | stfips + year 

--- Verificación de variables ---
¿Todas las variables están en df_es?
[1] "d_16" "d_17" "d_18" "d_19" "d_20"

--- Construcción de fórmula ---

✓ Fórmula final construida:
asmrs ~ d_-7 + d_-6 + d_-5 + d_-4 + d_-3 + d_-2 + d_0 + d_1 + d_2 + d_3 + d_4 + d_5 + d_6 + d_7 + d_8 + d_9 + d_10 + d_11 + d_12 + d_13 + d_14 + d_15 + d_16 + d_17 + d_18 + d_19 + d_20 | stfips + year 

--- Verificación de variables ---
¿Todas las variables están en df_es?
✓ 

ERROR: Error in feols(as.formula(formula_str), data = df_es, cluster = "stfips"): The variable 'd_' is in the RHS (first part) of the formula but not in the
data set.
Maybe you meant: `d_-7`, `d_-6`, `d_-5`, `d_-4`, `d_-3`, `d_-2`, `d_-1`,
`d_0`, `d_1` or 19 others?


## d) Visualización del Event-Study

In [ ]:
plot_data <- es_results_df %>%
  mutate(
    pre_treatment = if_else(event_time < 0, 1, 0)
  )

p <- plot_data %>%
  ggplot(aes(x = event_time, y = estimate, color = factor(pre_treatment))) +
  # Zona de pre-tratamiento sombreada (gris claro)
  annotate(
    "rect",
    xmin = LOWER_BOUND - 0.5,
    xmax = -0.5,
    ymin = -Inf,
    ymax = Inf,
    fill = "lightgray",
    alpha = 0.2,
    color = NA
  ) +
  # Zona de post-tratamiento sombreada (muy claro)
  annotate(
    "rect",
    xmin = -0.5,
    xmax = UPPER_BOUND + 0.5,
    ymin = -Inf,
    ymax = Inf,
    fill = "lightyellow",
    alpha = 0.15,
    color = NA
  ) +
  # Línea en cero
  geom_hline(yintercept = 0, linetype = "dashed", color = "gray50", linewidth = 0.5) +
  # Línea en x = -0.5 (separación pre/post)
  geom_vline(xintercept = -0.5, linetype = "dashed", color = "darkred", linewidth = 0.4, alpha = 0.6) +
  # Barras de error
  geom_errorbar(
    aes(ymin = ci_lower, ymax = ci_upper),
    width = 0.2,
    position = position_dodge(0),
    linewidth = 0.6
  ) +
  # Puntos
  geom_point(size = 2.5, position = position_dodge(0)) +
  # Línea conectando puntos
  geom_line(aes(group = 1), linewidth = 0.5, alpha = 0.7) +
  # Etiquetas y temas
  scale_color_manual(
    values = c("0" = "#2E86AB", "1" = "#A23B72"),
    labels = c("0" = "Post-tratamiento", "1" = "Pre-tratamiento"),
    name = "Período"
  ) +
  labs(
    title = "Event-Study: Efectos dinámicos del tratamiento sobre ASMRS",
    subtitle = "Con intervalos de confianza al 95% (líneas), períodos pre/post sombreados",
    x = "Tiempo relativo al tratamiento (años)",
    y = "Efecto sobre ASMRS (coeficiente)",
    caption = "Período de referencia: t = -1. Barras de error: ±1.96 × SE con clustering en stfips."
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 14, face = "bold", hjust = 0.5),
    plot.subtitle = element_text(size = 11, hjust = 0.5, color = "gray40"),
    axis.title = element_text(size = 11, face = "bold"),
    legend.position = "top",
    panel.grid.major.x = element_blank(),
    panel.grid.minor = element_blank()
  )

print(p)

ggsave("../output/salida_3.png", plot = p, width = 12, height = 6, dpi = 300)
cat("\n✓ Gráfico del event-study guardado en: ../output/salida_3.png\n")

## Resumen de resultados

### Salidas generadas:
- `salida_1.csv`: Tabla de frecuencias del tiempo relativo al tratamiento
- `salida_2.csv`: Coeficientes del event-study con intervalos de confianza al 95%
- `salida_3.png`: Gráfico del event-study con visualización de períodos pre/post-tratamiento

### Interpretación:
El event-study estima efectos dinámicos del tratamiento, permitiendo verificar:
1. **Pre-tendencias**: Los coeficientes pre-tratamiento (t < -1) deben ser cercanos a cero e insignificantes
2. **Efecto promedio**: El coeficiente t = 0 (año de adopción) representa el impacto en el primer año
3. **Evolución temporal**: Los coeficientes posteriores (t > 0) muestran cómo evoluciona el efecto en el tiempo

### Especificación de la regresión:
$$ASMRS_{it} = \beta_0 + \sum_{k=\text{LOWER}}^{\text{UPPER}} \beta_k D^k_{it} + \gamma_i + \delta_t + \epsilon_{it}$$

donde:
- $D^k_{it}$ = indicadores de tiempo relativo (excluida la categoría k = -1)
- $\gamma_i$ = efectos fijos de entidad (stfips)
- $\delta_t$ = efectos fijos de tiempo (year)
- $\epsilon_{it}$ = término de error con clustering a nivel de entidad